<a href="https://colab.research.google.com/github/dippatel1994/Google-Tunix-Hackathon-Dip/blob/main/setup_tunix_gemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tunix Gemma Setup
This notebook prepares a Gemma checkpoint for Tunix-based supervised fine-tuning and GRPO experiments on Kaggle TPUs.

## Goals
- Install Tunix and required libraries on the TPU runtime.
- Authenticate with Hugging Face to download Gemma checkpoints.
- Load tokenizer and model in Flax/JAX with bf16 precision.
- Scaffold Tunix supervised and GRPO trainers ready for custom datasets and rewards.

In [1]:
#start from a fresh runtime to remove conflicting wheels
!pip install --quiet --upgrade pip setuptools wheel

#lock JAX + numpy/numba to TPU-friendly builds
!pip install --quiet "numpy==2.1.2" "numba<0.62" "jax==0.4.30" "jaxlib==0.4.30"

#install Tunix and friends on top
!pip install --quiet "google-tunix[prod]" flax==0.8.2 transformers==4.45.2 accelerate datasets sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.2/488.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.3/504.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.6/91.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━

In [2]:
# Cell 2 — Imports and runtime checks
import os
import random
import jax
import jax.numpy as jnp
from transformers import AutoTokenizer, FlaxAutoModelForCausalLM
import tunix


SEED = 42
random.seed(SEED)
jax.random.key(SEED)

devices = jax.devices()
print(f"Detected {len(devices)} JAX device(s).")
print(devices[0])

/usr/local/lib/python3.12/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.7.2 is installed, but it is not compatible with the installed jaxlib version 0.8.1, so it will not be used.
  warnings.warn(


ImportError: cannot import name '_center' from 'numpy._core.umath' (/usr/local/lib/python3.12/dist-packages/numpy/_core/umath.py)

In [ ]:
# Cell 3 — Hugging Face authentication helpers
import getpass

HF_TOKEN = os.environ.get("HF_TOKEN")
if not HF_TOKEN:
    HF_TOKEN = getpass.getpass("Enter your Hugging Face token (read access only): ")

os.environ["HF_TOKEN"] = HF_TOKEN
os.environ["HF_HOME"] = os.environ.get("HF_HOME", "/kaggle/temp/hf-cache")

MODEL_NAME = "google/gemma-2-2b-it"  # swap to base Gemma variant if desired
TOKENIZER_NAME = MODEL_NAME
print(f"Using checkpoint: {MODEL_NAME}")

In [ ]:
# Cell 4 — Load tokenizer and base model
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME, token=HF_TOKEN)
tokenizer.padding_side = "right"
tokenizer.truncation_side = "left"

model = FlaxAutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    token=HF_TOKEN,
    dtype=jnp.bfloat16,
    _do_init=False,
)

param_count = sum(p.size for p in jax.tree_util.tree_leaves(model.params))
print(f"Loaded model with ~{param_count / 1e6:.1f}M parameters in bf16.")

In [ ]:
# Cell 5 — Tunix supervised fine-tuning scaffold
from datasets import load_dataset
from tunix.training.supervised import SupervisedConfig, SupervisedTrainer

DATASET_NAME = "gsm8k"
dataset = load_dataset(DATASET_NAME, "main", split="train[:128]")

def format_example(example):
    prompt = f"Question: {example['question']}\nPlease think step by step before answering."
    answer = example["answer"].split("####")[-1].strip()
    target = f"<reasoning>{example['answer']}</reasoning><answer>{answer}</answer>"
    return {"prompt": prompt, "response": target}

processed_dataset = dataset.map(format_example, remove_columns=dataset.column_names)

sft_config = SupervisedConfig(
    learning_rate=1e-5,
    max_steps=10,
    per_device_batch_size=1,
    gradient_accumulation_steps=4,
    max_seq_length=768,
    output_dir="/kaggle/working/tunix-sft-checkpoints",
    logging_steps=1,
 )

sft_trainer = SupervisedTrainer(
    model=model,
    tokenizer=tokenizer,
    config=sft_config,
    train_dataset=processed_dataset,
 )

print("Supervised trainer initialized — ready for warmup runs.")

In [ ]:
# Cell 6 — GRPO configuration preview
from tunix.training.rl import GRPOConfig, GRPOTrainer
from tunix.rewards import basic_trace_reward

grpo_config = GRPOConfig(
    learning_rate=5e-6,
    kl_weight=0.1,
    num_generations=4,
    max_prompt_length=512,
    max_response_length=512,
    total_training_steps=50,
    logging_steps=5,
 )

grpo_trainer = GRPOTrainer(
    model=model,
    tokenizer=tokenizer,
    reward_fn=basic_trace_reward,
    config=grpo_config,
    prompt_dataset=processed_dataset,
 )

print("GRPO trainer scaffold ready — plug in custom rewards before training.")

## Next Steps
- Replace the placeholder GSM8k slice with the custom curated dataset.
- Swap in the process-aware reward function before launching GRPO training.
- Promote this notebook to Kaggle once verified end-to-end on TPU.